# Image Collections, Bands, and Pixels (Beginner Lab)

## Libraries and datasets (quick reference)
1. This notebook uses Earth Engine and geemap.
2. Dataset: Landsat 9 surface reflectance (`LANDSAT/LC09/C02/T1_L2`).

![Placeholder: A raster grid with pixel cells and stacked bands.]()


In [13]:
# Install the libraries we need.
!pip install -q --upgrade earthengine-api geemap


In [14]:
# Import libraries for Earth Engine and mapping.
import ee  # Earth Engine API
import geemap  # Map display helper

# Authenticate and initialize Earth Engine.
try:
    ee.Initialize()  # Use saved credentials if available
except Exception:
    ee.Authenticate()  # Launch login if needed
    ee.Initialize()  # Retry initialization

# Create a map for visualization.
Map = geemap.Map()  # Interactive map


## What is an ImageCollection?

An ImageCollection is a time-ordered stack of satellite images. We can filter it by location, date, or quality.


In [15]:
# Load the Landsat 9 image collection.
stanford = ee.Geometry.Point([-122.1697, 37.4275])  # Point geometry for Stanford University

collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').filterBounds(stanford)  # Landsat 9 dataset filtered to Stanford

# Print the first image to show its metadata structure.
collectionInfo = collection.first().getInfo()  # Get metadata of the first image

collectionInfo # Display the metadata of the first image in the collection


{'type': 'Image',
 'bands': [{'id': 'SR_B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7801, 7751],
   'crs': 'EPSG:32610',
   'crs_transform': [30, 0, 448485, 0, -30, 4263915]},
  {'id': 'SR_B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7801, 7751],
   'crs': 'EPSG:32610',
   'crs_transform': [30, 0, 448485, 0, -30, 4263915]},
  {'id': 'SR_B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7801, 7751],
   'crs': 'EPSG:32610',
   'crs_transform': [30, 0, 448485, 0, -30, 4263915]},
  {'id': 'SR_B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7801, 7751],
   'crs': 'EPSG:32610',
   'crs_transform': [30, 0, 448485, 0, -30, 4263915]},
  {'id': 'SR_B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
 

## Using Metadata



In [16]:
# Extract and display the footprint geometry from the image metadata.
# The footprint is stored in properties['system:footprint'] as a GeoJSON geometry.
# Extract the first image from the collection (matches the metadata displayed above).
image = collection.first()

footprint = image.get('system:footprint')  # Get the footprint property

# Convert the footprint to an Earth Engine Geometry for display.
footprint_geometry = ee.Geometry(footprint)  # Wrap as EE Geometry

# Zoom the map to the footprint extent.
Map.centerObject(footprint_geometry, 8)  # Center and zoom to footprint

# Add the footprint boundary to the map.
Map.addLayer(footprint_geometry, {'color': 'yellow'}, 'Image Footprint')

# Display the map with the footprint overlay.
Map

Map(center=[37.47635190662506, -122.25297452264654], controls=(WidgetControl(options=['position', 'transparent…

## Bands and Pixels

A single satellite image contains multiple bands (like layers). Each pixel stores a value per band. In this case, our image also carries other bands, including the ST_CDIST band, which gives the distance to clouds. Here, we demonstrate teh manipulation/selection of pixels, within a band, by selecting only the pixels that are equal to 0, or are "cloudy"


In [17]:
# Clear all layers except the basemap
for layer in Map.layers[1:]:
    Map.remove_layer(layer)
    
# Display only ST_CDIST pixels equal to 0 as a blue cloud mask.
cloud_mask = image.select('ST_CDIST').eq(0).selfMask()
Map.addLayer(cloud_mask, {'palette': ['0000FF']}, 'Cloud Mask (ST_CDIST = 0)')

# Display the map.
Map


Map(center=[37.47635190662506, -122.25297452264654], controls=(WidgetControl(options=['position', 'transparent…

# Interactions Between Bands and Band Math


In [18]:
# Clear all layers except the basemap
for layer in Map.layers[1:]:
    Map.remove_layer(layer)

# Calculate NDVI from the image
red = image.select('SR_B4')
nir = image.select('SR_B5')
ndvi = nir.subtract(red).divide(nir.add(red)).updateMask(image.select('ST_CDIST').gt(0))  # Mask out clouds using ST_CDIST

# Set visualization parameters
ndvi_vis = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}

# Add NDVI layer to map
Map.addLayer(ndvi, ndvi_vis, 'NDVI (Cloud Masked)')

# Display the map
Map


Map(center=[37.47635190662506, -122.25297452264654], controls=(WidgetControl(options=['position', 'transparent…

# Creating RGB Imagery Visualizations

In [19]:
# Clear all layers except the basemap
for layer in Map.layers[1:]:
    Map.remove_layer(layer)

# Create False Color IR-R-G composite
nir = image.select('SR_B5')    # Near-infrared
red = image.select('SR_B4')    # Red
green = image.select('SR_B3')  # Green

false_color_ir_r_g = nir.addBands(red).addBands(green)

# Set visualization parameters for False Color
false_color_vis = {'min': 0, 'max': 15000, 'bands': ['SR_B5', 'SR_B4', 'SR_B3']}

# Add False Color layer to map
Map.addLayer(false_color_ir_r_g, false_color_vis, 'False Color (IR-R-G)')

# Display the map
Map

Map(center=[37.47635190662506, -122.25297452264654], controls=(WidgetControl(options=['position', 'transparent…

# Dynamic Visualizations

In [20]:
# Extract the first image from the collection
image = collection.first()

# Define stretch percentage (adjust this value and rerun the cell to see different stretches)
stretch_percent = 2  # Try 1, 2, or 5 for different contrast

# Clear all layers except the basemap
for layer in Map.layers[1:]:
    Map.remove_layer(layer)

# Create False Color IR-R-G composite
nir = image.select('SR_B5')    # Near-infrared
red = image.select('SR_B4')    # Red
green = image.select('SR_B3')  # Green

false_color_ir_r_g = nir.addBands(red).addBands(green)

# Helper function for dynamic stretch (same pattern as the multispectral lab)
def get_dynamic_vis(image, bands, region, scale=90, stretch=2):
    # Reduce to percentiles for each band.
    stats = image.select(bands).reduceRegion(
        reducer=ee.Reducer.percentile([stretch, 100 - stretch]),
        geometry=region,
        scale=scale,
        maxPixels=1e9
    ).getInfo()

    # Collect per-band min and max values.
    mins = []
    maxs = []
    for b in bands:
        low_key = f'{b}_p{stretch}'
        high_key = f'{b}_p{100 - stretch}'
        if stats.get(low_key) is not None:
            mins.append(stats[low_key])
        if stats.get(high_key) is not None:
            maxs.append(stats[high_key])

    # Fallback if stats are missing.
    if not mins or not maxs:
        return {'min': 0, 'max': 15000, 'bands': bands}

    return {
        'min': min(mins),
        'max': max(maxs),
        'bands': bands
    }

# Build dynamic visualization parameters.
false_color_vis = get_dynamic_vis(
    false_color_ir_r_g,
    ['SR_B5', 'SR_B4', 'SR_B3'],
    footprint_geometry,
    scale=90,
    stretch=stretch_percent
)

# Add False Color layer to map
Map.addLayer(false_color_ir_r_g, false_color_vis, 'False Color (IR-R-G, Dynamic Stretch)')

# Display the map
Map


Map(center=[37.47635190662506, -122.25297452264654], controls=(WidgetControl(options=['position', 'transparent…